In [1]:
import pandas as pd
import re
import numpy as np
from datetime import datetime

attrac_df = pd.read_csv('Attraction_Data\Attraction_Tripadvisor_Data_SEA.csv')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18304\909019521.py:6: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  attrac_df = pd.read_csv('Attraction_Data\Attraction_Tripadvisor_Data_SEA.csv')


In [2]:
attrac_df.duplicated().sum()

0

In [3]:
attrac_df.drop_duplicates()

,Attraction ID,Country,Tourist Attraction,Name,Address,Website,Phone Number,Category,Ranking,Opening Hour,...,Review Count,Excellent Reviews,Very Good Reviews,Average Reviews,Poor Reviews,Terrible Reviews,Near Restaurants,Near Attractions,URL,Image URL
0,AT000001,Vietnam,Ha Giang,Ma Pi Leng Pass,NaN,NaN,NaN,['Scenic Drives'],#2 of 8 things to do in Ha Giang,12:00 AM - 11:59 PM,...,226,207.0,15.0,3.0,1.0,0.0,NaN,NaN,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...
1,AT000002,Vietnam,Ha Giang,Dong Van Karst Plateau Geopark,"Sa Phin, Ha Giang 310000 Vietnam",NaN,NaN,['Geologic Formations'],#1 of 8 things to do in Ha Giang,12:00 AM - 11:59 PM,...,233,214.0,16.0,3.0,0.0,0.0,1,2,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...
2,AT000003,Vietnam,Ha Giang,Thon Tha Village,"Thon Tha Village, Phuong Do Commune Thon Tha V...",NaN,NaN,"['Neighborhoods', 'Points of Interest & Landma...",#4 of 8 things to do in Ha Giang,8:00 AM - 12:00 AM,...,67,64.0,2.0,0.0,1.0,0.0,50,1,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...
3,AT000004,Vietnam,Ha Giang,Lung Cu Flag Tower,"Dragon Mountain Lung Cu Peak, Ha Giang 29000 V...",http://www.hagiangtrekkingtour.com/tours/hagia...,%2B84%2090%20486%2059%2035,"['Points of Interest & Landmarks', 'Observatio...",#3 of 8 things to do in Ha Giang,NaN,...,173,81.0,48.0,37.0,5.0,2.0,3,NaN,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...
4,AT000005,Vietnam,Ha Giang,Nho Que River,NaN,NaN,NaN,['Bodies of Water'],#2 of 5 things to do in Dong Van,NaN,...,5,5.0,0.0,0.0,0.0,0.0,NaN,NaN,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72805,AT072806,Singapore,Singapore,Johor Transport,"44 Toh Tuck Road Goodluck Garden #01-07, Singa...",http://www.johortransport.com/,%2B65%209632%203700,['Taxis & Shuttles'],#337 of 348 Transportation in Singapore,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"1,047",344.0,https://www.tripadvisor.com/Attraction_Review-...,NaN
72806,AT072807,Singapore,Singapore,Angel Foot Reflexology,118 Rivervale Drive Rivervale Plaza Rivervale ...,http://www.angelfoot.com.sg/,%2B65%206635%202785,['Spas'],#641 of 675 Spas & Wellness in Singapore,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,575,246.0,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...
72807,AT072808,Singapore,Singapore,Precious Foot Reflexology,"446 Clementi Avenue 3 #01-197, Singapore 12044...",http://www.preciousfoot.com.sg/,%2B65%208363%204703,['Spas'],#638 of 675 Spas & Wellness in Singapore,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"1,059",499.0,https://www.tripadvisor.com/Attraction_Review-...,NaN
72808,AT072809,Singapore,Singapore,Ashi Foot Reflexology,"61 Ang Mo Kio Avenue 8 #02-06 Jubilee Square, ...",http://ashifoot.com/,%2B65%206456%207588,['Spas'],#635 of 675 Spas & Wellness in Singapore,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,968,651.0,https://www.tripadvisor.com/Attraction_Review-...,https://dynamic-media-cdn.tripadvisor.com/medi...


In [4]:
def missingCounter(df):
    missing_data = df.isna().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending = False )
    print(missing_data )
    return missing_data.index
missingCounter(attrac_df)

Price                72658
Description          56568
Address              46466
Duration             45741
Opening Hour         39651
Near Restaurants     37781
Near Attractions     36806
Image URL            36456
Email                28800
Review Count         19388
Rating               19388
Excellent Reviews    19376
Very Good Reviews    19376
Average Reviews      19376
Poor Reviews         19376
Terrible Reviews     19376
Website              19307
Phone Number         17219
Category               257
Ranking                248
dtype: int64


Index(['Price', 'Description', 'Address', 'Duration', 'Opening Hour',
       'Near Restaurants', 'Near Attractions', 'Image URL', 'Email',
       'Review Count', 'Rating', 'Excellent Reviews', 'Very Good Reviews',
       'Average Reviews', 'Poor Reviews', 'Terrible Reviews', 'Website',
       'Phone Number', 'Category', 'Ranking'],
      dtype='object')

In [5]:
# Xóa các dòng có giá trị NaN trong cột 'Name'
attrac_df = attrac_df.dropna(subset=['Name'])


In [6]:
attrac_df['Address'] = attrac_df.apply(
    lambda row: row['Tourist Attraction'] + ", " + row['Country'] if pd.isnull(row['Address']) else row['Address'],
    axis=1
)

In [7]:
import unicodedata
# Xóa các dòng có giá trị NaN trong cột 'Name'
attrac_df = attrac_df.dropna(subset=['Name'])

# Hàm chuyển chuỗi có dấu sang không dấu
def remove_diacritics(text):
    if pd.isna(text): return ''
    text = unicodedata.normalize('NFD', str(text))
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    text = text.replace('Đ', 'D').replace('đ', 'd')  # Xử lý đặc biệt
    return text


# Làm sạch cột 'Name'
def clean_name(text):
    if pd.isna(text): return ''
    text = remove_diacritics(text)
    text = text.replace('Unclaimed', '').strip()
    return text

# Làm sạch địa chỉ: bỏ dấu + bỏ mã số đứng trước tên quốc gia
def clean_address(text):
    if pd.isna(text): return ''
    text = remove_diacritics(text)

    # Bước 1: Xóa mã vùng kiểu "20000", "310000", v.v. trước tên quốc gia
    pattern = r'\s\d{4,6}(?=\s*(Vietnam|Thailand|Philippines|Malaysia|Singapore|Indonesia|Cambodia|Laos|Myanmar|Brunei|Timor))'
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Bước 2: Thêm dấu "," nếu không có dấu phẩy ngay trước tên quốc gia
    text = re.sub(r'(?<![,])\s+(?=(Vietnam|Thailand|Philippines|Malaysia|Singapore|Indonesia|Cambodia|Laos|Myanmar|Brunei|Timor)$)', r', ', text.strip(), flags=re.IGNORECASE)

    return text.strip()


attrac_df['Name'] = attrac_df['Name'].apply(clean_name)
attrac_df['Address'] = attrac_df['Address'].apply(clean_address)


In [8]:
attrac_df['Website'] = attrac_df['Website'].fillna('No website')
attrac_df['Phone Number'] = attrac_df['Phone Number'].fillna('No phone number')
attrac_df['Description'] = attrac_df['Description'].fillna('No description available')
attrac_df['Email'] = attrac_df['Email'].fillna('No email')


In [9]:
def clean_category_string(category_str):
    # Loại bỏ [, ], ', và khoảng trắng thừa
    if isinstance(category_str, str):
        category_str = re.sub(r"[\[\]']", "", category_str)
        return category_str.strip()
    return category_str

def extract_category_from_ranking_final_v2(ranking, category):
    """
    Ưu tiên lấy category từ Ranking.
    Nếu không có hoặc chỉ là 'things to do', thì lấy category đầu tiên từ Category.
    """
    if pd.isna(ranking) or ranking.lower() in ['nan', 'unknown', 'unranked', '']:
        return clean_category_string(extract_first_category(category))
    # Regex lấy phần giữa 'of <số> ' và ' in '
    match = re.search(r'of\s+\d+\s+(.*?)\s+in\s', ranking, flags=re.IGNORECASE)
    if match:
        category_raw = match.group(1).strip().lower()
        category_raw = clean_category_string(category_raw)
        if category_raw == 'things to do':
            return clean_category_string(extract_first_category(category))
        else:
            return category_raw
    else:
        return clean_category_string(extract_first_category(category))

def extract_first_category(category):
    if pd.isna(category) or category.lower() == 'unknown':
        return 'unknown'
    # Loại bỏ [, ], '
    category = re.sub(r"[\[\]']", "", category)
    categories = [cat.strip().lower() for cat in category.split(',') if cat.strip()]
    return categories[0] if categories else 'unknown'

# Áp dụng vào DataFrame
attrac_df['Category_Type'] = attrac_df.apply(
    lambda row: extract_category_from_ranking_final_v2(row['Ranking'], row['Category']),
    axis=1
)

In [10]:
attrac_df = attrac_df.drop(columns=['Category', 'Image URL'])


In [11]:
from datetime import datetime


def fill_default_opening_hour(opening_hour):
    if pd.isna(opening_hour) or str(opening_hour).lower() in ['nan', 'none', '', 'unknown']:
        return "8:00 AM - 5:30 PM"
    return opening_hour

# Áp dụng điền mặc định
attrac_df['Opening Hour'] = attrac_df['Opening Hour'].apply(fill_default_opening_hour)

# Hàm tách giờ mở và giờ đóng (kết quả dạng string 24h 'HH:MM')
def extract_open_close_time(opening_hour):
    # Tìm 2 time bằng regex: đầu và cuối chuỗi
    match = re.match(r'([0-9]{1,2}:[0-9]{2}\s*[APMapm]{2})\s*-\s*([0-9]{1,2}:[0-9]{2}\s*[APMapm]{2})', opening_hour)
    if match:
        open_str = match.group(1)
        close_str = match.group(2)
        # Chuyển về 24h để tiện phân tích
        open_time = datetime.strptime(open_str.strip().upper(), "%I:%M %p").strftime("%H:%M")
        close_time = datetime.strptime(close_str.strip().upper(), "%I:%M %p").strftime("%H:%M")
        return open_time, close_time
    return (np.nan, np.nan)

def calc_duration(open_time, close_time):
    try:
        t1 = datetime.strptime(open_time, "%H:%M")
        t2 = datetime.strptime(close_time, "%H:%M")
        duration = (t2 - t1).total_seconds() / 3600
        if duration < 0:
            duration += 24
        # Làm tròn thông thường
        return int(round(duration))
    except:
        return np.nan
    
    
attrac_df[['Opening_Time', 'Closing_Time']] = attrac_df['Opening Hour'].apply(
    lambda x: pd.Series(extract_open_close_time(x))
)
# Rồi mới được tính duration
attrac_df['Open_Hour_Duration'] = attrac_df.apply(
    lambda row: calc_duration(row['Opening_Time'], row['Closing_Time']),
    axis=1
)




In [12]:
attrac_df['Opening_Time'] = attrac_df['Opening_Time'].astype(str).replace("nan", "00:00")
attrac_df['Closing_Time'] = attrac_df['Closing_Time'].astype(str).replace("nan", "00:00")
attrac_df['Open_Hour_Duration'] = attrac_df['Open_Hour_Duration'].fillna(0).astype(int)


In [13]:
def bucket_opening_hours(open_time, close_time):
    # Xử lý trường hợp thiếu
    if pd.isna(open_time) or pd.isna(close_time):
        return pd.Series([False]*5, 
                         index=['open_morning', 'open_noon', 'open_afternoon', 'open_evening', 'open_night'])
    t_open = int(open_time.split(':')[0]) + int(open_time.split(':')[1])/60
    t_close = int(close_time.split(':')[0]) + int(close_time.split(':')[1])/60
    # Xử lý qua ngày
    if t_close < t_open:
        t_close += 24

    # Check từng bucket
    def overlaps(start, end):
        # start/end: dạng float giờ (0-24)
        # Xét nếu attraction mở cửa giao với khung giờ này
        # Nếu open qua đêm thì normalize t_close > 24
        return not (t_close <= start or t_open >= end)

    # Khung giờ
    morning = overlaps(6, 12)
    noon = overlaps(12, 14)
    afternoon = overlaps(14, 18)
    evening = overlaps(18, 22)
    night = overlaps(22, 24) or overlaps(0, 6)
    return pd.Series([morning, noon, afternoon, evening, night], 
                     index=['open_morning', 'open_noon', 'open_afternoon', 'open_evening', 'open_night'])

# Áp dụng vào DataFrame (dùng các cột đã chuẩn hóa ở trên)
attrac_df[['open_morning', 'open_noon', 'open_afternoon', 'open_evening', 'open_night']] = attrac_df.apply(
    lambda row: bucket_opening_hours(row['Opening_Time'], row['Closing_Time']),
    axis=1
)



In [14]:
attrac_df = attrac_df.drop(columns=['Opening Hour', 'Opening_Time', 'Closing_Time'])


In [15]:
attrac_df['Duration'] = attrac_df['Duration'].fillna('No duration')

In [16]:
review_cols = [
    'Review Count', 'Excellent Reviews', 'Very Good Reviews',
    'Average Reviews', 'Poor Reviews', 'Terrible Reviews'
]

for col in review_cols:
    attrac_df[col] = pd.to_numeric(attrac_df[col], errors='coerce').fillna(0).astype(int)
    
attrac_df['Rating'] = attrac_df['Rating'].fillna(-1)

attrac_df['Rating'] = pd.to_numeric(attrac_df['Rating'], errors='coerce').astype(float)

# 2. Tính toán review_score
def calc_review_score(row):
    total_reviews = row['Excellent Reviews'] + row['Very Good Reviews'] + row['Average Reviews'] + row['Poor Reviews'] + row['Terrible Reviews']
    if total_reviews == 0:
        return 0  # hoặc -1 nếu bạn thích
    score = (
        5 * row['Excellent Reviews'] +
        4 * row['Very Good Reviews'] +
        3 * row['Average Reviews'] +
        2 * row['Poor Reviews'] +
        1 * row['Terrible Reviews']
    ) / total_reviews
    return round(score, 2)

attrac_df['review_score'] = attrac_df.apply(calc_review_score, axis=1)

In [17]:
near_cols = ['Near Restaurants', 'Near Attractions']

for col in near_cols:
    attrac_df[col] = pd.to_numeric(attrac_df[col], errors='coerce').fillna(0).astype(int)


In [18]:
attrac_df.to_csv('Attraction_Data\Attraction_Tripadvisor_Data_SEA_cleanedforweb.csv', index=False, encoding='utf-8', quoting=1)